# Création d'une base de référence d'agregats

In [1]:
# default_exp aggregates_read

In [2]:
# Activate multi-output in notebook
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

In [3]:
# export
import unittest

import yaml

from leximpact_socio_fisca_simu_etat.config import Configuration
from leximpact_socio_fisca_simu_etat.logger import logger

config = Configuration()
tc = unittest.TestCase()

## Lecture des agregats

In [4]:
# export

with open(config.get("AGREGATS")) as file:
    try:
        agregats = yaml.safe_load(file)
    except yaml.YAMLError as exc:
        msg = f"Error reading yaml file: {exc}"
        logger.error(msg)
        raise exc

In [5]:
agregats["2018"]["csg_deductible_salaire"]

{'ERFS': {'factor_to_2020': 1.1326784664772125,
  'factor_to_2021': 1.185625946449572,
  'factor_to_2022': 1.185625946449572}}

In [6]:
# export


def get_aggregate(year, variable_name, source, variable_type):
    try:
        return agregats[year][variable_name][source][variable_type]
    except Exception:
        # logger.debug(f"Error reading {year}.{variable_name}.{variable_type}: {e}") #  as e
        return None

In [7]:
tc.assertEqual(get_aggregate("2020", "csg_salaire", "ccss", "sum"), 72628000000)
tc.assertAlmostEqual(
    get_aggregate("2018", "csg_deductible_salaire", "ERFS", "factor_to_2022"),
    1.1856,
    places=2,
)
tc.assertAlmostEqual(
    get_aggregate("2018", "csg_imposable_salaire", "ERFS", "factor_to_2022"),
    1.1856,
    places=2,
)

In [8]:
tc.assertEqual(get_aggregate("2018", "retraites", "POTE", "sum"), 299949516289)
tc.assertEqual(get_aggregate("2018", "retraites", "POTE", "nexistepas"), None)
tc.assertEqual(get_aggregate("2018", "nexistepas", "POTE", "sum"), None)
tc.assertEqual(get_aggregate("nexistepas", "retraites", "POTE", "sum"), None)